### Matrix Multiplication
* load mnis
* matrix multiply 3 for loops in python - naive
* matrix multiply 2 for loops in python - elementwise multiplication
* matrix multiply 1 for loops in python - broadcasting
* matrix multiply 0 for loops in python - Einsein summation
* call PyTorch matmul()

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [32]:
#export
from exp.nb_00 import *
import operator

def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

def test_eq(a,b): test(a,b,operator.eq,'==')

In [36]:
#export
def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)
def test_near(a,b): test(a,b,near)

## Get data

In [1]:
#export
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor
from fastai.basics import *

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [2]:
path = datasets.download_data(MNIST_URL, ext='.gz'); path

PosixPath('/home/CW01/uia94835/.fastai/data/mnist.pkl.gz')

In [5]:
path = Config().data_path()/'mnist'
path

PosixPath('/home/CW01/uia94835/.fastai/data/mnist')

In [6]:
with gzip.open(path/'mnist.pkl.gz', 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [10]:
x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))
n,c = x_train.shape
x_train, x_train.shape, y_train, y_train.shape, y_train.min(), y_train.max()

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([50000, 784]),
 tensor([5, 0, 4,  ..., 8, 4, 8]),
 torch.Size([50000]),
 tensor(0),
 tensor(9))

### Variables setup

In [16]:
m1 = x_valid[:5]
weights = torch.randn(784, 10)
bias = torch.randn(10)
print(m1.shape)
print(weights.shape)
print(bias.shape)

torch.Size([5, 784])
torch.Size([784, 10])
torch.Size([10])


In [67]:
m2 = weights

### Matmul naive - 3 for loops

In [30]:
def matmul_3(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(br):
                c[i,j] += a[i,k] * b[k,j]
    return c

In [38]:
res3 = matmul_3(m1, m2)
res3.shape

torch.Size([5, 10])

In [39]:
test_near(res3, torch.matmul(m1, m2))

In [58]:
%time res3 = matmul_3(m1, m2)

CPU times: user 787 ms, sys: 0 ns, total: 787 ms
Wall time: 786 ms


In [217]:
%timeit -n 1000 _ = torch.matmul(m1, m2)

4.93 µs ± 269 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Matmul - 2 for loops - elementwise ops

In [186]:
def matmul_2(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i,j] = (a[i,:] * b[:,j]).sum()
    return c

In [187]:
res2 = matmul_2(m1, m2)
res2.shape

torch.Size([5, 10])

In [188]:
test_near(res2, torch.matmul(m1, m2))

In [201]:
%timeit -n 1000 res2 = matmul_2(m1, m2)

960 µs ± 128 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [204]:
960 / 1.56

615.3846153846154

### Matmul - 1 for loops - Broadcasting

In [205]:
def matmul_1(a,b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        c[i] = (a[i,:][:,None] * b).sum(dim=0)
    return c

In [206]:
res1 = matmul_1(m1, m2)
res1.shape

torch.Size([5, 10])

In [207]:
test_near(res1, torch.matmul(m1, m2))

In [208]:
%timeit -n 1000 res1 = matmul_1(m1, m2)

192 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [209]:
787/0.19

4142.105263157895

### Matmul - 0 for loops - Einstein summation

In [210]:
def matmul_0(a,b):
#     ar, ac = a.shape
#     br, bc = b.shape
#     assert ac == br
    return torch.einsum('ik,kj->ij',a,b)

In [211]:
res0 = matmul_0(m1, m2)
res0.shape

torch.Size([5, 10])

In [212]:
test_near(res0, torch.matmul(m1, m2))

In [213]:
%timeit -n 1000 res0 = matmul_0(m1, m2)

24.8 µs ± 539 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [215]:
787/0.025

31480.0

In [218]:
787/ 0.005

157400.0

### PyTorch operation shorthand

In [219]:
res_pt = m1@m2
res_pt.shape

torch.Size([5, 10])